In [ ]:
!pip install opencv-python-headless

In [ ]:
import cv2
import numpy as np

In [21]:
class LaneDetector:
    # Constructor de la clase LaneDetector
    def __init__(self):
        pass

    # Método para aplicar una máscara en la región de interés de la imagen
    def region_of_interest(self, img, vertices):
        # Crear una máscara negra del mismo tamaño que la imagen
        mask = np.zeros_like(img)
        # Definir el color de la máscara que coincide con la región de interés
        match_mask_color = 255
        # Rellenar la máscara con el color en la región de interés
        cv2.fillPoly(mask, vertices, match_mask_color)
        # Aplicar la máscara a la imagen
        masked_image = cv2.bitwise_and(img, mask)
        # Devolver la imagen con la máscara aplicada
        return masked_image

    # Método para dibujar líneas en la imagen
    def draw_lines(self, img, lines):
        # Crear una copia de la imagen
        img = np.copy(img)
        # Crear una imagen negra del mismo tamaño que la imagen original
        line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
        # Recorrer todas las líneas
        for line in lines:
            # Recorrer todos los puntos de la línea
            for x1, y1, x2, y2 in line:
                # Dibujar la línea en la imagen de líneas
                cv2.line(line_img, (x1, y1), (x2, y2), (0, 255, 0), 3)
        # Combinar la imagen original con la imagen de líneas
        img = cv2.addWeighted(img, 0.8, line_img, 1, 0)
        # Devolver la imagen con las líneas dibujadas
        return img

    # Método para procesar un fotograma (frame) de video y detectar líneas
    def process_frame(self, img):
        # Convertir la imagen a escala de grises
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Aplicar desenfoque Gaussiano a la imagen en escala de grises
        img_blur = cv2.GaussianBlur(gray_img, (3, 3), 0, 0)
        # Detectar bordes usando el algoritmo de Canny
        img_canny = cv2.Canny(img_blur, 40, 120)
        # Obtener las dimensiones de la imagen
        height, width = img.shape[:2]
        # Definir los vértices de la región de interés
        vertices = np.array([[(width*0.1, height), (width*0.4, height*0.6), (width*0.6, height*0.6), (width*0.9, height)]], dtype=np.int32)
        # Aplicar la región de interés a la imagen con bordes detectados
        img_roi = self.region_of_interest(img_canny, vertices)

        # Definir parámetros para la transformada de Hough
        rho = 2
        theta = np.pi / 180
        threshold = 40
        min_line_len = 50
        max_line_gap = 10
        # Aplicar la transformada de Hough para detectar líneas
        lines = cv2.HoughLinesP(img_roi, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)

        if lines is not None:
            img_with_lines = self.draw_lines(img, lines)
        else:
            img_with_lines = img

        return img_with_lines

    # Método para procesar un video completo y detectar líneas en cada fotograma (frame)
    def process_video(self, input_video_path, output_video_path):
        # Leer el video de entrada
        cap = cv2.VideoCapture(input_video_path)
        # Obtener las dimensiones y la tasa de fotogramas del video
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = int(cap.get(cv2.CAP_PROP_FPS))

        # Definir el códec y crear el objeto VideoWriter para el video de salida
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

        # Leer y procesar cada fotograma (frame) del video de entrada
        while cap.isOpened():
            ret, frame = cap.read()
            if ret:
                # Procesar el fotograma actual y detectar líneas
                result_frame = self.process_frame(frame)
                # Escribir el fotograma procesado en el video de salida
                out.write(result_frame)
                # Mostrar el fotograma procesado en una ventana
                cv2.imshow('Processed Frame', result_frame)

                # Si se presiona la tecla 'q', interrumpir el procesamiento del video
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            # Si no hay más fotogramas para leer, terminar el procesamiento
            else:
                break

        # Liberar recursos y cerrar ventanas
        cap.release()
        out.release()
        cv2.destroyAllWindows()

input_video_path = 'video1.mp4'
output_video_path = 'output_video_1.mp4'
lane_detector = LaneDetector()
lane_detector.process_video(input_video_path, output_video_path)
